In [1]:
# API jupyter notebook file

In [2]:
# Dependencies
import requests
import json
import pandas as pd
from pprint import pprint
import numpy as np

# World bank base url
url = "http://api.worldbank.org/v2/country/?format=json"

# create empty list to hold all countries
all_countries = []
income_levels = []
country_codes = []
lats = []
lngs = []
regions = []

# loop through 6 pages of countries, adding countries to our empty list as we go
for x in range(6):
    countries_response = requests.get(f"{url}&page={x+1}").json() 
    #import pdb; pdb.set_trace()
    countries = countries_response[1]
    for country in countries:
        all_countries.append(country["name"])
        income_levels.append(country["incomeLevel"]['value'])
        country_codes.append(country['id'])
        lats.append(country['latitude'])
        lngs.append(country['longitude'])
        regions.append(country['region']['value'])

# print lengths of lists to check validity
print(f'Length of countries: {len(all_countries)}\nLength of income_levels: {len(income_levels)}\nLength of country_code: {len(country_codes)}')
print(f'Length of lats: {len(lats)}\nLength of lngs: {len(lngs)}\nLength of lats: {len(lats)}\nLength of lngs: {len(lngs)}')
print(f'Length of regions: {len(regions)}')
countries_df = pd.DataFrame({
    "Country" : all_countries,
    "Income Lvl" : income_levels,
    "Country Code" : country_codes,
    "Lat" : lats,
    "Lng" : lngs,
    "Region" : regions
})

clean_df = countries_df.loc[countries_df["Region"] != "Aggregates", :]

clean_df.to_csv("clean_countries.csv", index=False)

clean_df.head(5)

len(clean_df)

Length of countries: 297
Length of income_levels: 297
Length of country_code: 297
Length of lats: 297
Length of lngs: 297
Length of lats: 297
Length of lngs: 297
Length of regions: 297


218

In [3]:
years = [2015, 2016,2017]

country_codes_test = clean_df['Country Code'][0:4]

for year in years:
    for code in country_codes_test:
        print(year,code)
        query_url = "http://api.worldbank.org/v2/country/" + code + "/indicator/NY.GDP.MKTP.CD?format=json&date=" + str(year)

2015 ABW
2015 AFG
2015 AGO
2015 ALB
2016 ABW
2016 AFG
2016 AGO
2016 ALB
2017 ABW
2017 AFG
2017 AGO
2017 ALB


In [6]:
# Practice some calls using specific countries... total population


def yr_calls(year):
    """function changes year in request call
    Args:
        year(int):This is the year for api call 
    Returns:
        gpds(list): This returns yearly gpd for each country code
    """
#     final_countries = []
#     final_codes = []
    gdps = []

    for code in clean_df["Country Code"]:

        #Build Query Url
        try:
            query_url = "http://api.worldbank.org/v2/country/" + code + "/indicator/NY.GDP.MKTP.CD?format=json&date=" + str(year) 
            response = requests.get(query_url).json()
            #import pdb; pdb.set_trace()
            gdps.append(response[1][0]["value"])
#             final_countries.append(response[1][0]['country']['value'])
#             final_codes.append(response[1][0]['countryiso3code'])


        except:
            print(f"{code}'s GDP was not found")
            gdps.append("0")
#             final_countries.append("0")
#             final_codes.append("0")


    print(json.dumps(response, indent=4))
    
    return gdps

In [7]:
# Using yr_calls function to return 2015 GDP as gdp_2015(list)

gdp_2015 = yr_calls(2015)

TWN's GDP was not found
[
    {
        "page": 1,
        "pages": 1,
        "per_page": 50,
        "total": 1,
        "sourceid": "2",
        "sourcename": "World Development Indicators",
        "lastupdated": "2021-04-26"
    },
    [
        {
            "indicator": {
                "id": "NY.GDP.MKTP.CD",
                "value": "GDP (current US$)"
            },
            "country": {
                "id": "ZW",
                "value": "Zimbabwe"
            },
            "countryiso3code": "ZWE",
            "date": "2015",
            "value": 19963120600,
            "unit": "",
            "obs_status": "",
            "decimal": 0
        }
    ]
]


In [8]:
#creating gdp dataframes
            
clean_df['2015 GDP'] = gdp_2015
clean_df.head()

C:\Users\jonat\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Country,Income Lvl,Country Code,Lat,Lng,Region,2015 GDP
0,Aruba,High income,ABW,12.5167,-70.0167,Latin America & Caribbean,2.91955e+09
1,Afghanistan,Low income,AFG,34.5228,69.1761,South Asia,1.99071e+10
3,Angola,Lower middle income,AGO,-8.81155,13.242,Sub-Saharan Africa,1.16194e+11
4,Albania,Upper middle income,ALB,41.3317,19.8172,Europe & Central Asia,1.13868e+10
5,Andorra,High income,AND,42.5075,1.5218,Europe & Central Asia,2.78987e+09


In [9]:
# Using yr_calls function to return 2018 GDP as gdp_2018(list)
gdp_2016 = yr_calls(2016)

TWN's GDP was not found
[
    {
        "page": 1,
        "pages": 1,
        "per_page": 50,
        "total": 1,
        "sourceid": "2",
        "sourcename": "World Development Indicators",
        "lastupdated": "2021-04-26"
    },
    [
        {
            "indicator": {
                "id": "NY.GDP.MKTP.CD",
                "value": "GDP (current US$)"
            },
            "country": {
                "id": "ZW",
                "value": "Zimbabwe"
            },
            "countryiso3code": "ZWE",
            "date": "2016",
            "value": 20548678100,
            "unit": "",
            "obs_status": "",
            "decimal": 0
        }
    ]
]


In [10]:
# Adding gdp_YR(list) to clean_df 
clean_df['2016 GDP'] = gdp_2016
clean_df.head()

C:\Users\jonat\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Country,Income Lvl,Country Code,Lat,Lng,Region,2015 GDP,2016 GDP
0,Aruba,High income,ABW,12.5167,-70.0167,Latin America & Caribbean,2.91955e+09,2.96592e+09
1,Afghanistan,Low income,AFG,34.5228,69.1761,South Asia,1.99071e+10,1.80177e+10
3,Angola,Lower middle income,AGO,-8.81155,13.242,Sub-Saharan Africa,1.16194e+11,1.01124e+11
4,Albania,Upper middle income,ALB,41.3317,19.8172,Europe & Central Asia,1.13868e+10,1.18612e+10
5,Andorra,High income,AND,42.5075,1.5218,Europe & Central Asia,2.78987e+09,2.89668e+09


In [11]:
# Using yr_calls function to return 2017 GDP as gdp_2017(list)
gdp_2017 = yr_calls(2017)

TWN's GDP was not found
[
    {
        "page": 1,
        "pages": 1,
        "per_page": 50,
        "total": 1,
        "sourceid": "2",
        "sourcename": "World Development Indicators",
        "lastupdated": "2021-04-26"
    },
    [
        {
            "indicator": {
                "id": "NY.GDP.MKTP.CD",
                "value": "GDP (current US$)"
            },
            "country": {
                "id": "ZW",
                "value": "Zimbabwe"
            },
            "countryiso3code": "ZWE",
            "date": "2017",
            "value": 22040902300,
            "unit": "",
            "obs_status": "",
            "decimal": 0
        }
    ]
]


In [12]:
# Adding gdp_YR(list) to clean_df 
clean_df['2017 GDP'] = gdp_2017
clean_df.head()

C:\Users\jonat\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Country,Income Lvl,Country Code,Lat,Lng,Region,2015 GDP,2016 GDP,2017 GDP
0,Aruba,High income,ABW,12.5167,-70.0167,Latin America & Caribbean,2.91955e+09,2.96592e+09,3.05642e+09
1,Afghanistan,Low income,AFG,34.5228,69.1761,South Asia,1.99071e+10,1.80177e+10,1.88699e+10
3,Angola,Lower middle income,AGO,-8.81155,13.242,Sub-Saharan Africa,1.16194e+11,1.01124e+11,1.22124e+11
4,Albania,Upper middle income,ALB,41.3317,19.8172,Europe & Central Asia,1.13868e+10,1.18612e+10,1.30197e+10
5,Andorra,High income,AND,42.5075,1.5218,Europe & Central Asia,2.78987e+09,2.89668e+09,3.00018e+09


In [13]:
# Using yr_calls function to return 2018 GDP as gdp_2018(list)
gdp_2018 = yr_calls(2018)

TWN's GDP was not found
[
    {
        "page": 1,
        "pages": 1,
        "per_page": 50,
        "total": 1,
        "sourceid": "2",
        "sourcename": "World Development Indicators",
        "lastupdated": "2021-04-26"
    },
    [
        {
            "indicator": {
                "id": "NY.GDP.MKTP.CD",
                "value": "GDP (current US$)"
            },
            "country": {
                "id": "ZW",
                "value": "Zimbabwe"
            },
            "countryiso3code": "ZWE",
            "date": "2018",
            "value": 24311560500,
            "unit": "",
            "obs_status": "",
            "decimal": 0
        }
    ]
]


In [14]:
# Adding gdp_YR(list) to clean_df 
clean_df['2018 GDP'] = gdp_2018
clean_df.head()

C:\Users\jonat\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Country,Income Lvl,Country Code,Lat,Lng,Region,2015 GDP,2016 GDP,2017 GDP,2018 GDP
0,Aruba,High income,ABW,12.5167,-70.0167,Latin America & Caribbean,2.91955e+09,2.96592e+09,3.05642e+09,None
1,Afghanistan,Low income,AFG,34.5228,69.1761,South Asia,1.99071e+10,1.80177e+10,1.88699e+10,1.83539e+10
3,Angola,Lower middle income,AGO,-8.81155,13.242,Sub-Saharan Africa,1.16194e+11,1.01124e+11,1.22124e+11,1.01353e+11
4,Albania,Upper middle income,ALB,41.3317,19.8172,Europe & Central Asia,1.13868e+10,1.18612e+10,1.30197e+10,1.5147e+10
5,Andorra,High income,AND,42.5075,1.5218,Europe & Central Asia,2.78987e+09,2.89668e+09,3.00018e+09,3.21832e+09


In [15]:
# Using yr_calls function to return 2019 GDP as gdp_2019(list)
gdp_2019 = yr_calls(2019)

TWN's GDP was not found
[
    {
        "page": 1,
        "pages": 1,
        "per_page": 50,
        "total": 1,
        "sourceid": "2",
        "sourcename": "World Development Indicators",
        "lastupdated": "2021-04-26"
    },
    [
        {
            "indicator": {
                "id": "NY.GDP.MKTP.CD",
                "value": "GDP (current US$)"
            },
            "country": {
                "id": "ZW",
                "value": "Zimbabwe"
            },
            "countryiso3code": "ZWE",
            "date": "2019",
            "value": 21440758800,
            "unit": "",
            "obs_status": "",
            "decimal": 0
        }
    ]
]


In [17]:
# Adding gdp_YR(list) to clean_df 
clean_df['2019 GDP'] = gdp_2019
clean_df = clean_df.fillna(0)
clean_df.head()

,Country,Income Lvl,Country Code,Lat,Lng,Region,2015 GDP,2016 GDP,2017 GDP,2018 GDP,2019 GDP
0,Aruba,High income,ABW,12.5167,-70.0167,Latin America & Caribbean,2.91955e+09,2.96592e+09,3.05642e+09,0,0
1,Afghanistan,Low income,AFG,34.5228,69.1761,South Asia,1.99071e+10,1.80177e+10,1.88699e+10,1.83539e+10,1.92911e+10
3,Angola,Lower middle income,AGO,-8.81155,13.242,Sub-Saharan Africa,1.16194e+11,1.01124e+11,1.22124e+11,1.01353e+11,8.88157e+10
4,Albania,Upper middle income,ALB,41.3317,19.8172,Europe & Central Asia,1.13868e+10,1.18612e+10,1.30197e+10,1.5147e+10,1.52792e+10
5,Andorra,High income,AND,42.5075,1.5218,Europe & Central Asia,2.78987e+09,2.89668e+09,3.00018e+09,3.21832e+09,3.15406e+09


# Adam's Section

# Cheyenne Section

# Nathan Section

# Jonathan Section

In [33]:
j_url = "http://api.worldbank.org/v2/indicator?format=json"
indicator_response = requests.get(j_url).json()
indicator_names = []
for x in range(50):
    indicator_names.append(indicator_response[1][x]['name'])
#indicator_names.index('Poverty Gap ($1.90 a day)')
indicator_names

['Poverty Headcount ($1.90 a day)',
 'Poverty Headcount ($2.50 a day)',
 'Middle Class ($10-50 a day) Headcount',
 'Official Moderate Poverty Rate-National',
 'Poverty Headcount ($4 a day)',
 'Vulnerable ($4-10 a day) Headcount',
 'Poverty Gap ($1.90 a day)',
 'Poverty Gap ($2.50 a day)',
 'Poverty Gap ($4 a day)',
 'Poverty Severity ($1.90 a day)',
 'Poverty Severity ($2.50 a day)',
 'Poverty Severity ($4 a day)',
 'Poverty Headcount ($1.90 a day)-Rural',
 'Poverty Headcount ($2.50 a day)-Rural',
 'Middle Class ($10-50 a day) Headcount-Rural',
 'Official Moderate Poverty Rate- Rural',
 'Poverty Headcount ($4 a day)-Rural',
 'Vulnerable ($4-10 a day) Headcount-Rural',
 'Poverty Gap ($1.90 a day)-Rural',
 'Poverty Gap ($2.50 a day)-Rural',
 'Poverty Gap ($4 a day)-Rural',
 'Poverty Severity ($1.90 a day)-Rural',
 'Poverty Severity ($2.50 a day)-Rural',
 'Poverty Severity ($4 a day)-Rural',
 'Access to electricity (% of total population)',
 'Total final energy consumption (TFEC)',
 'Lite

In [184]:
# define empty lists to append data to
populations = []
    
# run for loop, looping throug hall the country (codes) in our Countries DataFrame (named clean_df)
for code in clean_df["Country Code"]:

    #Build Query Url
    try:
        population_url = "http://api.worldbank.org/v2/country/" + code + "/indicator/SP.POP.TOTL?format=json" 
        pop_response = requests.get(population_url).json()
        #import pdb; pdb.set_trace()
            
        # append population values to list
        populations.append(int(pop_response[1][1]['value']))

    except (KeyError, TypeError):
        print(f"{code}'s Population Data was not found")
        populations.append(0)


ERI's Population Data was not found
TWN's Population Data was not found


In [186]:
# create new DataFrame to add populations
population_df = clean_df.copy()

# turn populations list into populations(100,000s) by dividing by a 100,000
divided_pops = [pop / 100000 for pop in populations]

# add population data to DataFrame
population_df["2019 Population (1000s)"] = populations

population_df


,Country,Income Lvl,Country Code,Lat,Lng,Region,2015 GDP,2016 GDP,2017 GDP,2018 GDP,2019 GDP,2019 Population (1000s)
0,Aruba,High income,ABW,12.5167,-70.0167,Latin America & Caribbean,2.91955e+09,2.96592e+09,3.05642e+09,0,0,106314
1,Afghanistan,Low income,AFG,34.5228,69.1761,South Asia,1.99071e+10,1.80177e+10,1.88699e+10,1.83539e+10,1.92911e+10,38041754
3,Angola,Lower middle income,AGO,-8.81155,13.242,Sub-Saharan Africa,1.16194e+11,1.01124e+11,1.22124e+11,1.01353e+11,8.88157e+10,31825295
4,Albania,Upper middle income,ALB,41.3317,19.8172,Europe & Central Asia,1.13868e+10,1.18612e+10,1.30197e+10,1.5147e+10,1.52792e+10,2854191
5,Andorra,High income,AND,42.5075,1.5218,Europe & Central Asia,2.78987e+09,2.89668e+09,3.00018e+09,3.21832e+09,3.15406e+09,77142
...,...,...,...,...,...,...,...,...,...,...,...,...
291,Kosovo,Upper middle income,XKX,42.565,20.926,Europe & Central Asia,6.44292e+09,6.71917e+09,7.24571e+09,7.94296e+09,7.92613e+09,1788878
293,"Yemen, Rep.",Low income,YEM,15.352,44.2075,Middle East & North Africa,4.24506e+10,3.0936e+10,2.67361e+10,2.34863e+10,2.25811e+10,29161922
294,South Africa,Upper middle income,ZAF,-25.746,28.1871,Sub-Saharan Africa,3.17621e+11,2.96357e+11,3.49554e+11,3.68289e+11,3.51432e+11,58558267
295,Zambia,Lower middle income,ZMB,-15.3982,28.2937,Sub-Saharan Africa,2.12433e+10,2.09548e+10,2.58682e+10,2.63121e+10,2.33098e+10,17861030


In [ ]:
[{'page': 1, 'pages': 370, 'per_page': '50', 'total': 18489},
 [{'id': '1.0.HCount.1.90usd',
   'name': 'Poverty Headcount ($1.90 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2011 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.HCount.2.5usd',
   'name': 'Poverty Headcount ($2.50 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.HCount.Mid10to50',
   'name': 'Middle Class ($10-50 a day) Headcount',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.HCount.Ofcl',
   'name': 'Official Moderate Poverty Rate-National',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income below the official poverty line developed by each country.',
   'sourceOrganization': 'LAC Equity Lab tabulations of data from National Statistical Offices.',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.HCount.Poor4uds',
   'name': 'Poverty Headcount ($4 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.HCount.Vul4to10',
   'name': 'Vulnerable ($4-10 a day) Headcount',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.PGap.1.90usd',
   'name': 'Poverty Gap ($1.90 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.PGap.2.5usd',
   'name': 'Poverty Gap ($2.50 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.PGap.Poor4uds',
   'name': 'Poverty Gap ($4 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.PSev.1.90usd',
   'name': 'Poverty Severity ($1.90 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.PSev.2.5usd',
   'name': 'Poverty Severity ($2.50 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.0.PSev.Poor4uds',
   'name': 'Poverty Severity ($4 a day)',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.HCount.1.90usd',
   'name': 'Poverty Headcount ($1.90 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2011 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.HCount.2.5usd',
   'name': 'Poverty Headcount ($2.50 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.HCount.Mid10to50',
   'name': 'Middle Class ($10-50 a day) Headcount-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.HCount.Ofcl',
   'name': 'Official Moderate Poverty Rate- Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income below the official poverty line developed by each country.',
   'sourceOrganization': 'LAC Equity Lab tabulations of data from National Statistical Offices.',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.HCount.Poor4uds',
   'name': 'Poverty Headcount ($4 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.HCount.Vul4to10',
   'name': 'Vulnerable ($4-10 a day) Headcount-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.PGap.1.90usd',
   'name': 'Poverty Gap ($1.90 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.PGap.2.5usd',
   'name': 'Poverty Gap ($2.50 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.PGap.Poor4uds',
   'name': 'Poverty Gap ($4 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.PSev.1.90usd',
   'name': 'Poverty Severity ($1.90 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.PSev.2.5usd',
   'name': 'Poverty Severity ($2.50 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1.PSev.Poor4uds',
   'name': 'Poverty Severity ($4 a day)-Rural',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.1_ACCESS.ELECTRICITY.TOT',
   'name': 'Access to electricity (% of total population)',
   'unit': '',
   'source': {'id': '35', 'value': 'Sustainable Energy for All'},
   'sourceNote': 'Access to electricity is the percentage of population with access to electricity.',
   'sourceOrganization': 'World Bank Global Electrification Database 2012',
   'topics': []},
  {'id': '1.1_TOTAL.FINAL.ENERGY.CONSUM',
   'name': 'Total final energy consumption (TFEC)',
   'unit': '',
   'source': {'id': '35', 'value': 'Sustainable Energy for All'},
   'sourceNote': 'Total final energy consumption (TFEC): This indicator is derived form energy balances statistics and is equivalent to total final consumption excluding non-energy use.',
   'sourceOrganization': 'World Bank and International Energy Agency (IEA Statistics © OECD/IEA, http://www.iea.org/stats/index.asp).  ',
   'topics': []},
  {'id': '1.1_YOUTH.LITERACY.RATE',
   'name': 'Literacy rate, youth total (% of people ages 15-24) ',
   'unit': '',
   'source': {'id': '34', 'value': 'Global Partnership for Education'},
   'sourceNote': 'The number of persons aged 15 to 24 years who can both read and write with understanding a short simple statement on their everyday life, divided by the population in that age group. Generally, ‘literacy’ also encompasses ‘numeracy’, the ability to make simple arithmetic calculations. For further country-specific definition details please refer to the source of information, the UNESCO Institute for Statistics (UIS): www.uis.unesco.org',
   'sourceOrganization': 'Source of information: UNESCO Institute for Statistics (www.uis.unesco.org). Please refer to its website for country-specific details on the specific national data sources and method used. ',
   'topics': []},
  {'id': '1.2.HCount.1.90usd',
   'name': 'Poverty Headcount ($1.90 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2011 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.HCount.2.5usd',
   'name': 'Poverty Headcount ($2.50 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.HCount.Mid10to50',
   'name': 'Middle Class ($10-50 a day) Headcount-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.HCount.Ofcl',
   'name': 'Official Moderate Poverty Rate-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income below the official poverty line developed by each country.',
   'sourceOrganization': 'LAC Equity Lab tabulations of data from National Statistical Offices.',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.HCount.Poor4uds',
   'name': 'Poverty Headcount ($4 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.HCount.Vul4to10',
   'name': 'Vulnerable ($4-10 a day) Headcount-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty headcount index measures the proportion of the population with daily per capita income (in 2005 PPP) below the poverty line.',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.PGap.1.90usd',
   'name': 'Poverty Gap ($1.90 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.PGap.2.5usd',
   'name': 'Poverty Gap ($2.50 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.PGap.Poor4uds',
   'name': 'Poverty Gap ($4 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty gap captures the mean aggregate income or consumption shortfall relative to the poverty line across the entire population. It measures the total resources needed to bring all the poor to the level of the poverty line (averaged over the total population). ',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.PSev.1.90usd',
   'name': 'Poverty Severity ($1.90 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.PSev.2.5usd',
   'name': 'Poverty Severity ($2.50 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2.PSev.Poor4uds',
   'name': 'Poverty Severity ($4 a day)-Urban',
   'unit': '',
   'source': {'id': '37', 'value': 'LAC Equity Lab'},
   'sourceNote': 'The poverty severity index combines information on both poverty and inequality among the poor by averaging the squares of the poverty gaps relative the poverty line',
   'sourceOrganization': 'LAC Equity Lab tabulations of SEDLAC (CEDLAS and the World Bank).',
   'topics': [{'id': '11', 'value': 'Poverty '}]},
  {'id': '1.2_ACCESS.ELECTRICITY.RURAL',
   'name': 'Access to electricity (% of rural population)',
   'unit': '',
   'source': {'id': '35', 'value': 'Sustainable Energy for All'},
   'sourceNote': 'Access to electricity is the percentage of rural population with access to electricity. ',
   'sourceOrganization': 'World Bank Global Electrification Database 2013',
   'topics': []},
  {'id': '1.3_ACCESS.ELECTRICITY.URBAN',
   'name': 'Access to electricity (% of urban population)',
   'unit': '',
   'source': {'id': '35', 'value': 'Sustainable Energy for All'},
   'sourceNote': 'Access to electricity is the percentage of total population with access to electricity. ',
   'sourceOrganization': 'World Bank Global Electrification Database 2014',
   'topics': []},
  {'id': '100000',
   'name': '100000:GROSS DOMESTIC PRODUCT',
   'unit': '',
   'source': {'id': '71',
    'value': 'International Comparison Program (ICP) 2005'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '1000000',
   'name': '1000000:GROSS DOMESTIC PRODUCT',
   'unit': '',
   'source': {'id': '78', 'value': 'ICP 2017'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '110000',
   'name': '110000:INDIVIDUAL CONSUMPTION EXPENDITURE BY HOUSEHOLDS',
   'unit': '',
   'source': {'id': '71',
    'value': 'International Comparison Program (ICP) 2005'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '110100',
   'name': '110100:FOOD AND NON-ALCOHOLIC BEVERAGES',
   'unit': '',
   'source': {'id': '71',
    'value': 'International Comparison Program (ICP) 2005'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '1101000',
   'name': '1101000:FOOD AND NON-ALCOHOLIC BEVERAGES',
   'unit': '',
   'source': {'id': '78', 'value': 'ICP 2017'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '110110',
   'name': '110110:FOOD',
   'unit': '',
   'source': {'id': '71',
    'value': 'International Comparison Program (ICP) 2005'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '1101100',
   'name': '1101100:FOOD',
   'unit': '',
   'source': {'id': '78', 'value': 'ICP 2017'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '110111',
   'name': '110111:Bread and cereals (Class)',
   'unit': '',
   'source': {'id': '71',
    'value': 'International Comparison Program (ICP) 2005'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': []},
  {'id': '1101110',
   'name': '1101110:Bread and cereals',
   'unit': '',
   'source': {'id': '78', 'value': 'ICP 2017'},
   'sourceNote': '',
   'sourceOrganization': '',
   'topics': [{}]}]]